### **Assignment:** Conversation Management & Classification using Groq API

In [1]:
# Testing API Response

import os
import openai

client = openai.OpenAI(
    base_url = "https://api.groq.com/openai/v1",
    api_key = "my api key" # Not showing API on Github due to security issues but it is available on Colab Link I provided you.
)

In [2]:
try:
    response = client.chat.completions.create(
    model="openai/gpt-oss-20b", # selecting a valid Grok Model
    messages=[{"role": "user", "content": "hello"}]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print("Error:", e)

Hello! 👋 How can I help you today?


### **Task 1:** Managing Conversation History with Summarization

* We defined a Conversation_Manager class to maintain the ongoing chat history between user and assistant.
* To keep the context manageable and optimize performance, the history is truncated based on configurable limits (either by the number of conversation turns or total character length)
* Additionally, after every k-th turn (configurable), the conversation is summarized using the Groq GPT model (openai/gpt-oss-20b), replacing previous messages with a concise summary.

This approach balances maintaining relevant context while preventing uncontrolled growth of conversation data.

In [3]:
class Conversation_Manager:
    """
    Manages conversation history between user and assistant.
    Supports:
    - Adding new turns to history.
    - Truncation by max turns or max characters.
    - Periodic summarization after every k-th run.
    """
    def __init__(self, summarization_frequency=3, max_turns=None, max_chars=None):
        self.history = []
        self.run_count = 0
        self.summarization_frequency = summarization_frequency
        self.max_turns = max_turns
        self.max_chars = max_chars

    def add_message(self, role, content):  # appends a message dict. with role and text content to history
        self.history.append({"role": role, "content": content})

    def truncate_history(self):
        """
        - If max_turns specified, keeps only last max_turns messages.
        - If max_chars specified, removes oldest messages until total text length <= max_chars.
        """
        if self.max_turns and len(self.history) > self.max_turns:
            self.history = self.history[-self.max_turns:]
        if self.max_chars:
            total_len = sum(len(m['content']) for m in self.history)
            while total_len > self.max_chars and self.history:
                removed = self.history.pop(0)
                total_len -= len(removed['content'])

    def summarize_history(self):
        """
        this will combine all messages into a prompt, requests summary and then
        replaces history with single summary message.
        """
        prompt = "Summarize this conversation briefly:\n\n"
        for msg in self.history:
            prompt += f"{msg['role'].capitalize()}: {msg['content']}\n"

        response = client.chat.completions.create(
            model="openai/gpt-oss-20b",  # using a model which is compatible with Grok
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150,
            temperature=0.5,
        )
        summary = response.choices[0].message.content
        # Now this will replace entire history with this summary as a single message
        self.history = [{"role": "assistant", "content": summary}]
        return summary

    def process_new_turn(self, user_message, assistant_response):
        """
        - Process a new conversation turn: add messages, truncate, could possibly summarize.
        - Also summarizes after every k-th run, controlled by summarization_frequency.
        """
        self.run_count += 1
        self.add_message("user", user_message)
        self.add_message("assistant", assistant_response)
        self.truncate_history()

        if self.summarization_frequency and (self.run_count % self.summarization_frequency == 0):
            summary = self.summarize_history()
            return summary
        return None

    def get_history(self):
        return self.history

Final Demonstartion of Task 1 Functionality->

In [4]:
conv_mgr = Conversation_Manager(summarization_frequency=3, max_turns=6, max_chars=1000)

samples = [
    ("Hello! How are you?", "I am good, thank you! How can I help?"),
    ("Tell me about AI.", "AI is the simulation of human intelligence by machines."),
    ("What is OpenAI function calling?", "OpenAI function calling lets you define schemas for structured chat output."),
    ("Explain JSON schema.", "JSON schema defines the structure of JSON data."),
    ("Thanks!", "You're welcome!"),
]

for i, (user_msg, assistant_msg) in enumerate(samples, start=1):
    summary = conv_mgr.process_new_turn(user_msg, assistant_msg)
    print(f"Turn {i} added.")
    if summary:  # printing summary on every k-th turn(here 3rd turn)
        print(f"Summary after turn {i}:\n{summary}\n")

print("Final Conversation History:")
for msg in conv_mgr.get_history():
    print(f"{msg['role'].capitalize()}: {msg['content']}")

Turn 1 added.
Turn 2 added.
Turn 3 added.
Summary after turn 3:
The user greets the assistant, then asks two questions: one about what AI is and another about OpenAI’s function‑calling feature. The assistant replies with brief explanations for each topic.

Turn 4 added.
Turn 5 added.
Final Conversation History:
Assistant: The user greets the assistant, then asks two questions: one about what AI is and another about OpenAI’s function‑calling feature. The assistant replies with brief explanations for each topic.
User: Explain JSON schema.
Assistant: JSON schema defines the structure of JSON data.
User: Thanks!
Assistant: You're welcome!


**Task 1** is completed succesfully as above output fulfils all requirements of **Task 1** mentioned in Assignment



---



### **Task 2:** JSON Schema Classification & Information Extraction

* We created a JSON schema specifying key user information fields (name, email, phone, location, age) expected in the chat.
* Using Groq API’s OpenAI-compatible function calling, our code sends user chat samples and requests structured extraction of this information in JSON format following the schema.

Each extracted output is validated locally with jsonschema to ensure the data matches the expected types and constraints.

In [14]:
import jsonschema
import json

# JSON Schema for defining user info extraction structure

json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer", "minimum": 16, "maximum": 120}
    },
    "required": []
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_user_info",
            "description": "Extracts basic user information from chat.",
            "parameters": json_schema,
        },
    }
]

def call_function_calling_api(chat_text):
    """
    - Calls Groq API with OpenAI-compatible function calling.
    - Passes chat_text as input and requests structured extraction
      per the JSON schema.
    - Returns parsed JSON object from the function call response.
    """
    response = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=[{"role": "user", "content": chat_text}],
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "extract_user_info"}},
        temperature=0,
    )
    resp_message = response.choices[0].message

    if hasattr(resp_message, "tool_calls") and resp_message.tool_calls:
        tool_call = resp_message.tool_calls[0]
        try:
            func_args = json.loads(tool_call.function.arguments)
            return func_args
        except json.JSONDecodeError:
            print("Failed to decode function_call arguments")
            return None
    return None

def validate_output(data):
    """
    - Validates extracted JSON data against the predefined JSON schema.
    - Prints whether the output is valid or displays validation errors.
    """
    try:
        jsonschema.validate(instance=data, schema=json_schema)
        print("Output is valid JSON matching schema.")
    except jsonschema.ValidationError as e:
        print(f"Validation error: {e.message}")

In [15]:
# sample chats demonstrating data extraction and validation

sample_chats = [
    "Hi, my name is Aman, I am 30 years old and live in Delhi. My email is aman@example.com and phone number 9876543210.",
    "Hello, Priya here from Mumbai. Reach me at priya@gmail.com.",
    "Hey, I'm Rahul, aged 28. My contact is rahul28@mail.com.",
]

for chat in sample_chats:
    print(f"\nExtracting from chat: {chat}")
    extracted = call_function_calling_api(chat)
    if extracted:
        print("Extracted JSON:", extracted)
        validate_output(extracted)
    else:
        print("No extraction available.")


Extracting from chat: Hi, my name is Aman, I am 30 years old and live in Delhi. My email is aman@example.com and phone number 9876543210.
Extracted JSON: {'age': 30, 'email': 'aman@example.com', 'location': 'Delhi', 'name': 'Aman', 'phone': '9876543210'}
Output is valid JSON matching schema.

Extracting from chat: Hello, Priya here from Mumbai. Reach me at priya@gmail.com.
Extracted JSON: {'email': 'priya@gmail.com', 'location': 'Mumbai', 'name': 'Priya'}
Output is valid JSON matching schema.

Extracting from chat: Hey, I'm Rahul, aged 28. My contact is rahul28@mail.com.
Extracted JSON: {'age': 28, 'email': 'rahul28@mail.com'}
Output is valid JSON matching schema.


**Task 2** is also completed sucessfully.

Thank you for reviewing my assignment submission.

I have documented the code and logic clearly to reflect my approach and learning journey throughout this project.